In [16]:
from db_queries import get_population, get_ids
from db_queries import get_location_metadata as get_locs

In [1]:
import pandas as pd, numpy as np

In [2]:
gday_path =  '/ihme/homes/beatrixh/vivarium_data_analysis/pre_processing/lsff_project/data_prep/inputs/gday_extraction_sheet_02_19_2021.csv'
gday = pd.read_csv(gday_path)

In [3]:
gday.head()

,location_id,location_name,subnational_name,subnational_location_id,urbanicity,vehicle,nutrient,value_description,value_mean,value_025_percentile,value_975_percentile,sub_population,source_citation,source_link,source_year,source_type,notes,user,date_recorded
0,160.0,Afghanistan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,168.0,Angola,NaN,NaN,mixed/both,wheat flour,na,Median amount of vehicle consumed on previous ...,49,NaN,NaN,total population,NaN,https://www.nutritionintl.org/content/user_fil...,1997-2000,Survey - cross-sectional,Estimates of wheat flour and maize meal consum...,rohit,12/2/2020
2,168.0,Angola,NaN,NaN,mixed/both,maize flour,na,Median amount of vehicle consumed on previous ...,72,NaN,NaN,total population,NaN,https://www.nutritionintl.org/content/user_fil...,1997-2000,Survey - cross-sectional,Estimates of wheat flour and maize meal consum...,rohit,12/2/2020
3,168.0,Angola,NaN,NaN,mixed/both,wheat flour,na,Median amount of vehicle consumed on previous ...,89.32,NaN,NaN,total population,GFDx,https://fortificationdata.org/country-fortific...,2019,NaN,(grams/capita/day)Daily food availability,paulina,12/11/2020
4,168.0,Angola,NaN,NaN,mixed/both,maize flour,na,Median amount of vehicle consumed on previous ...,154.99,NaN,NaN,total population,GFDx,https://fortificationdata.org/country-fortific...,2019,NaN,(grams/capita/day)Daily food availability,paulina,12/11/2020


In [4]:
gday[gday.location_id.isna()].location_name.unique()

array(['China ', 'Egypt', 'Nepal', 'South Africa'], dtype=object)

In [5]:
gday[gday.location_name.isna()]

,location_id,location_name,subnational_name,subnational_location_id,urbanicity,vehicle,nutrient,value_description,value_mean,value_025_percentile,value_975_percentile,sub_population,source_citation,source_link,source_year,source_type,notes,user,date_recorded


In [6]:
gday.location_id = gday.location_id.fillna(-1).astype(int)

In [7]:
gday.location_name.unique()

array(['Afghanistan', 'Angola', 'Bangladesh', 'Burkina Faso',
       "Côte d'Ivoire", 'Cameroon', 'Chad', 'China ',
       'Democratic Republic of the Congo', 'Egypt', 'Ethiopia', 'Ghana',
       'India', 'Indonesia', 'Kenya', 'Madagascar', 'Mozambique',
       'Myanmar', 'Nepal', 'Niger', 'Nigeria', 'Pakistan', 'Philippines',
       'South Africa', 'Sudan', 'Uganda', 'United Republic of Tanzania',
       'Viet Nam', 'Yemen', 'Zambia'], dtype=object)

In [9]:
value_d_to_metric = {
 'Median amount of vehicle consumed on previous day among consumers (g/day)': 'median',
 'Mean per capita consumption (g/day)': 'mean',
 'Mean amount of vehicle consumed on previous day among consumers (g/day)': 'mean',
 'Mean micronutrient intake per capita (mg/day)': 'mean',
 'Daily per capita consumption (g)': 'CHECK',
 'Daily consumption (mg/d)': 'CHECK',
 'Consumption per person per day (g)': 'CHECK',
 'Estimated daily contribution from fortified foods (mg/d)': 'CHECK',
 'kg/capita/year': 'CHECK'
}

In [10]:
value_d_to_entity = {
 'Median amount of vehicle consumed on previous day among consumers (g/day)': 'vehicle',
 'Mean per capita consumption (g/day)': 'CHECK',
 'Mean amount of vehicle consumed on previous day among consumers (g/day)': 'vehicle',
 'Mean micronutrient intake per capita (mg/day)': 'nutrient',
 'Daily per capita consumption (g)': 'CHECK',
 'Daily consumption (mg/d)': 'CHECK',
 'Consumption per person per day (g)': 'CHECK',
 'Estimated daily contribution from fortified foods (mg/d)': 'CHECK',
 'kg/capita/year': 'CHECK'
}

In [11]:
value_d_to_mass_unit = {
 'Median amount of vehicle consumed on previous day among consumers (g/day)': 'g',
 'Mean per capita consumption (g/day)': 'g',
 'Mean amount of vehicle consumed on previous day among consumers (g/day)': 'g',
 'Mean micronutrient intake per capita (mg/day)': 'mg',
 'Daily per capita consumption (g)': 'g',
 'Daily consumption (mg/d)': 'mg',
 'Consumption per person per day (g)': 'g',
 'Estimated daily contribution from fortified foods (mg/d)': 'mg',
 'kg/capita/year': 'kg'
}

In [12]:
value_d_to_time_unit = {
 'Median amount of vehicle consumed on previous day among consumers (g/day)': 'day',
 'Mean per capita consumption (g/day)': 'day',
 'Mean amount of vehicle consumed on previous day among consumers (g/day)': 'day',
 'Mean micronutrient intake per capita (mg/day)': 'day',
 'Daily per capita consumption (g)': 'day',
 'Daily consumption (mg/d)': 'day',
 'Consumption per person per day (g)': 'day',
 'Estimated daily contribution from fortified foods (mg/d)': 'day',
 'kg/capita/year': 'year'
}

In [13]:
value_d_to_population = {
 'Median amount of vehicle consumed on previous day among consumers (g/day)': 'consumers',
 'Mean per capita consumption (g/day)': 'capita CHECK',
 'Mean amount of vehicle consumed on previous day among consumers (g/day)': 'consumers',
 'Mean micronutrient intake per capita (mg/day)': 'capita CHECK',
 'Daily per capita consumption (g)': 'capita CHECK',
 'Daily consumption (mg/d)': 'CHECK',
 'Consumption per person per day (g)': 'CHECK',
 'Estimated daily contribution from fortified foods (mg/d)': 'CHECK',
 'kg/capita/year': 'CHECK'
}

In [29]:
def format_value_d(df):
    df['metric'] = df.value_description.map(value_d_to_metric)
    df['entity'] = df.value_description.map(value_d_to_entity)
    df['mass_unit'] = df.value_description.map(value_d_to_mass_unit)
    df['time_unit'] = df.value_description.map(value_d_to_time_unit)
    df['pop_denom'] = df.value_description.map(value_d_to_population)
    
    return df

In [34]:
nigeria = gday[gday.location_name=="Nigeria"]

In [35]:
nigeria = format_value_d(nigeria)

/ihme/homes/beatrixh/.local/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/ihme/homes/beatrixh/.local/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/ihme/homes/beatrixh/.local/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try us

In [18]:
loc_metadata = get_locs(location_set_id=35, gbd_round_id=6, decomp_step="step4")

In [19]:
nigeria_subnat_ids = {
    'Ebonyi':25328,
    'Sokoto':25351
}

In [20]:
pop = get_population(age_group_id=22, 
                     location_id=list(nigeria_subnat_ids.values()),
                     year_id=2018,
                     sex_id=3,
                     gbd_round_id=6, 
                     decomp_step='step5')

In [21]:
pop

,age_group_id,location_id,year_id,sex_id,population,run_id
0,22,25328,2018,3,3.163671e+06,192
1,22,25351,2018,3,6.384879e+06,192


In [36]:
nigeria['subnational_id'] = nigeria.subnational_name.map(nigeria_subnat_ids)

/ihme/homes/beatrixh/.local/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [44]:
nigeria = nigeria.merge(pop[['location_id','population']].rename(columns={'location_id':'subnational_id'}), on = 'subnational_id', how = 'left')

In [45]:
nigeria.sort_values('vehicle')

,location_id,location_name,subnational_name,subnational_location_id,urbanicity,vehicle,nutrient,value_description,value_mean,value_025_percentile,...,notes,user,date_recorded,metric,entity,mass_unit,time_unit,pop_denom,subnational_id,population
6,214,Nigeria,Ebonyi,NaN,unknown,bouillon,na,Mean amount of vehicle consumed on previous da...,2,na,...,"in terms of consumption patterns, women of rep...",paulina,12/16/2020,mean,vehicle,g,day,consumers,25328.0,3.163671e+06
8,214,Nigeria,Sokoto,NaN,unknown,bouillon,na,Mean amount of vehicle consumed on previous da...,3,na,...,"in terms of consumption patterns, women of rep...",paulina,12/16/2020,mean,vehicle,g,day,consumers,25351.0,6.384879e+06
2,214,Nigeria,na,NaN,unknown,maize flour,na,Mean amount of vehicle consumed on previous da...,54,na,...,Estimates of wheat flour and maize meal consum...,paulina,12/11/2020,mean,vehicle,g,day,consumers,NaN,NaN
3,214,Nigeria,na,NaN,unknown,maize flour,na,Mean amount of vehicle consumed on previous da...,95.1,na,...,(grams/capita/day)Daily food availability,paulina,12/11/2020,mean,vehicle,g,day,consumers,NaN,NaN
12,214,Nigeria,NaN,NaN,NaN,maize flour,na,Mean per capita consumption (g/day),69.7,NaN,...,Estimated values calculated from FAO food bala...,beatrix,12/17/2020,mean,CHECK,g,day,capita CHECK,NaN,NaN
4,214,Nigeria,na,NaN,unknown,rice,na,Mean amount of vehicle consumed on previous da...,126.9,na,...,(grams/capita/day)Daily food availability,paulina,12/11/2020,mean,vehicle,g,day,consumers,NaN,NaN
7,214,Nigeria,Ebonyi,NaN,unknown,rice,na,Mean amount of vehicle consumed on previous da...,80,na,...,"in terms of consumption patterns, women of rep...",paulina,12/16/2020,mean,vehicle,g,day,consumers,25328.0,3.163671e+06
9,214,Nigeria,Sokoto,NaN,unknown,rice,na,Mean amount of vehicle consumed on previous da...,97,na,...,"in terms of consumption patterns, women of rep...",paulina,12/16/2020,mean,vehicle,g,day,consumers,25351.0,6.384879e+06
10,214,Nigeria,na,NaN,unknown,rice,na,Consumption per person per day (g),77,na,...,FAO. FAOSTAT. 2013 Food Balance Sheets. Intern...,paulina,12/29/2020,CHECK,CHECK,g,day,CHECK,NaN,NaN
5,214,Nigeria,na,NaN,unknown,salt,na,Mean amount of vehicle consumed on previous da...,7.2,na,...,(grams/capita/day)Daily food availability,paulina,12/11/2020,mean,vehicle,g,day,consumers,NaN,NaN


In [46]:
## we can population-weight bouillon

In [47]:
nigeria[nigeria.vehicle=="bouillon"]

,location_id,location_name,subnational_name,subnational_location_id,urbanicity,vehicle,nutrient,value_description,value_mean,value_025_percentile,...,notes,user,date_recorded,metric,entity,mass_unit,time_unit,pop_denom,subnational_id,population
6,214,Nigeria,Ebonyi,NaN,unknown,bouillon,na,Mean amount of vehicle consumed on previous da...,2,na,...,"in terms of consumption patterns, women of rep...",paulina,12/16/2020,mean,vehicle,g,day,consumers,25328.0,3.163671e+06
8,214,Nigeria,Sokoto,NaN,unknown,bouillon,na,Mean amount of vehicle consumed on previous da...,3,na,...,"in terms of consumption patterns, women of rep...",paulina,12/16/2020,mean,vehicle,g,day,consumers,25351.0,6.384879e+06


In [49]:
nigeria.columns

Index(['location_id', 'location_name', 'subnational_name',
       'subnational_location_id', 'urbanicity', 'vehicle', 'nutrient',
       'value_description', 'value_mean', 'value_025_percentile',
       'value_975_percentile', 'sub_population', 'source_citation',
       'source_link', 'source_year', 'source_type', 'notes', 'user',
       'date_recorded', 'metric', 'entity', 'mass_unit', 'time_unit',
       'pop_denom', 'subnational_id', 'population'],
      dtype='object')

In [50]:
view_cols = [
    'location_id', 'location_name',
       'subnational_location_id', 'urbanicity', 'vehicle', 'nutrient',
       'value_description', 'value_mean', 'value_025_percentile',
       'value_975_percentile', 'sub_population', 'source_citation',
       'source_link', 'source_year', 'source_type', 'notes'
]

nigeria.loc[nigeria.vehicle=="maize flour",view_cols] 

,location_id,location_name,subnational_location_id,urbanicity,vehicle,nutrient,value_description,value_mean,value_025_percentile,value_975_percentile,sub_population,source_citation,source_link,source_year,source_type,notes
2,214,Nigeria,NaN,unknown,maize flour,na,Mean amount of vehicle consumed on previous da...,54,na,na,total population,NaN,https://www.nutritionintl.org/content/user_fil...,1997-2000,NaN,Estimates of wheat flour and maize meal consum...
3,214,Nigeria,NaN,unknown,maize flour,na,Mean amount of vehicle consumed on previous da...,95.1,na,na,total population,GFDx,https://fortificationdata.org/country-fortific...,2017,NaN,(grams/capita/day)Daily food availability
12,214,Nigeria,NaN,NaN,maize flour,na,Mean per capita consumption (g/day),69.7,NaN,NaN,total population,"Nuss, E. T., & Tanumihardjo, S. A. (2011). Qua...",https://doi.org/10.3945/an.110.000182,2007,estimated from FAO balance sheets,Estimated values calculated from FAO food bala...


In [55]:
nigeria.loc[nigeria.vehicle=="maize flour",['metric', 'entity', 'mass_unit', 'time_unit',
       'pop_denom','source_year']] 

,metric,entity,mass_unit,time_unit,pop_denom,source_year
2,mean,vehicle,g,day,consumers,1997-2000
3,mean,vehicle,g,day,consumers,2017
12,mean,CHECK,g,day,capita CHECK,2007


In [53]:
for i in nigeria.loc[nigeria.vehicle=="maize flour",view_cols].notes:
    print(i)
    print(" \n")

Estimates of wheat flour and maize meal consumption for each country are shown in Table 3.6. These estimates were made by taking 75%, a standard extraction rate, of the 1997-2000 average of wheat and maize consumption from the FAO food balance sheets, except for Latin American countries where the figures were provided by the milling industry. The maize consumption is given only for selected countries where maize is an important food staple. One should be cautious in using the maize data since some of the supply may go to non-food uses, such as brewing. These figures can be used as an initial guide to identify cereal fortification opportunities, to help calculate the amount of micronutrients to add to achieve a desired public health impact, and to estimate the impact of an existing fortification program. Some of these values have questionable accuracy and care should be taken in their application if not corroborated by other data.
 

(grams/capita/day)Daily food availability
 

Estimate